In [8]:
import os
import pytorch_lightning as pl
import torch
from model.model_interface import LLM
import torch.utils.data as tud
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import torch.nn.functional as F
from tqdm.notebook import tqdm

torch.set_float32_matmul_precision('medium')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

LLM Config

In [17]:
gpt2_local_dir = "/nvme/yangyuchen1/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8"
gpt2_xl_local_dir = "/nvme/yangyuchen1/huggingface/hub/models--gpt2-xl/snapshots/33cdb5c0db5423c1879b1b9f16c352988e8754a8"
llama_7b_local_dir = "/nvme/yangyuchen1/huggingface/hub/models--decapoda-research--llama-7b-hf/snapshots/5f98eefcc80e437ef68d457ad7bf167c2c6a1348"
llm_config = {
    "model_name": llama_7b_local_dir,
}
mt = LLM(**llm_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Hook Config

In [20]:
mt.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [19]:
hook_config = {
    "retain_output": True,
    "retain_input": False,
    "edit_output": None,
    "clone": False,
    "detach": False,
    "device": "cpu"
}
# gpt2 config
n_layer = mt.model.config.n_layer
blocks = mt.model.transformer.h
# llama config
n_layer = mt.model.config.n_layer
blocks = mt.model.transformer.h


mt.clear_hook()
for i in range(n_layer):
    mt.add_hook(module=blocks[i],name=f"block_{i}", **hook_config)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   "detach": False,                                                                        │
│    7 │   "device": "cpu"                                                                         │
│    8 }                                                                                           │
│ ❱  9 n_layer = mt.model.config.n_layer                                                           │
│   10 blocks = mt.model.transformer.h                                                             │
│   11                                                                                             │
│   12 mt.clear_hook()                                                                             │
│                                                                                                  │
│ /nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/transformers/configuration_u │
│ tils.py:260 in __getattribute__                                                                  │
│                                                                                                  │
│   257 │   def __getattribute__(self, key):                                                       │
│   258 │   │   if key != "attribute_map" and key in super().__getattribute__("attribute_map"):    │
│   259 │   │   │   key = super().__getattribute__("attribute_map")[key]                           │
│ ❱ 260 │   │   return super().__getattribute__(key)                                               │
│   261 │                                                                                          │
│   262 │   def __init__(self, **kwargs):                                                          │
│   263 │   │   # Attributes with defaults                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'LlamaConfig' object has no attribute 'n_layer'

Dataset Config

In [14]:
from dataset.knowns import Knowns

dst = Knowns("/nvme/yangyuchen1/coding/gpt_re/data", mt.tokenizer)
dl = DataLoader(dst, batch_size=128, collate_fn=dst.collate_fn)

Loaded dataset with 1209 elements


Trainer config

In [15]:
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "auto",
    "devices" : [5],
}
trainer = pl.Trainer(**trainer_config)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
res = trainer.predict(mt, dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Predicting: 0it [00:00, ?it/s]

In [18]:
mt.hooks[0].output

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 mt.hooks[0].output                                                                           │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range